# HTTP

В документе рассмотрены основные сценарии работы по протоколу HTTP с использованием библиотеки python requests.

In [3]:
import requests
import json

## HTTP

"The Hypertext Transfer Protocol (HTTP) is an application-level protocol for distributed, collaborative, hypermedia information systems."

### Особенности
- Используется для передачи данных тестовых данных в любом формате
- Работает без сохранения состояния
- Возжна работа как без сохранения сетевого соединения так и создание сессии
- Стандартынй порт TCP/IP - 80

### Методы HTTP

- GET - используется, когда необходимо получить данные от сервера без изменения его состояния.
- POST - используется, когда после выполунения запроса необходимо изменить состояние сервера, например записать сообщение в хранилище на сервере.
- HEAD - как GET, только без самого сообщения (ответ будет содержать только заголовки).
- PUT - заменить сожержимое в точке назначения на содержимое сообщения.
- DELETE - удалить содержимое точки назначения
- OPTIONS - получить параметры сервера (форматы, стандарты, версии протоколов, с которыми работает адресат)

## [Requests](http://docs.python-requests.org/en/master/  )

In [4]:
r = requests.get('http://httpbin.org')

In [7]:
r.text[:200] 

"<!DOCTYPE html>\n<html>\n<head>\n  <meta http-equiv='content-type' value='text/html;charset=utf8'>\n  <meta name='generator' value='Ronn/v0.7.3 (http://github.com/rtomayko/ronn/tree/0.7.3)'>\n  <title>http"

Таким образом, мы получили html-документ от web-сервера, тот самый, что интерпретируется браузером.

Попробуем теперь 

In [16]:
# метод DELETE по тому же адресу. 
r = requests.delete('http://httpbin.org')

In [12]:
r.text


'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>405 Method Not Allowed</title>\n<h1>Method Not Allowed</h1>\n<p>The method is not allowed for the requested URL.</p>\n'

Видим, что сервер не разрешил нам удалить свое содержимое.

## Responses

В предыдущем пункте мы уже поглядели немного на ответы на HTTP запросы. Они записаны в переменной r.

Каждый ответ снабжен трехзначным статус кодом XXX, где первая цифра обозначает классификацию статуса полученного сообщения: 

1 - информация

2 - успешно

3 - ожидание следующего действия 

4 - ошибка клиента

5 - обшибка сервера

Остальные два знака уодируют некоторую служебную информацию. Таким образом устройство работающее с протоколом HTTP не обязано понимать все статус коды, ему достаточно лишь знать свой необходимый минимум.

Посмотрим на статусы некоторых наших запросов:

In [22]:
r = requests.get('http://httpbin.org')
r.status_code  # все должно быть хорошо, ждем 200

200

In [23]:
r = requests.delete('http://httpbin.org')
r.status_code # мы явно накосячили, ждем 4**

405

## Headers

Заголовки это очень важно. Они описывают HTTP сообщения (их содержимое и свойства). 

Заголовки бывают

Общие (для клиента и сервера)

Клиентские (только для клиента)

Серверные (только для сервера)

мета-заголовки (про данные или сервер/клиент (если данных нет))

Подробнее можно почитать здесь https://www.tutorialspoint.com/http/http_header_fields.htm

Добавление заголовков к HTTP запросам в requests происходит с помощью параметра headers соответствующих функций, например:


In [28]:
r = requests.get('http://httpbin.org/get', headers={'header1':'val1'})
r.json()
# Видим наш кастомный заголовок среди всех прочих

{'args': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Header1': 'val1',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.12.4'},
 'origin': '92.242.59.6',
 'url': 'http://httpbin.org/get'}

Так же бывает удобно отправлять некоторые данные вида пар key:value, используя query string. В requests это делается очень просто, через параметр params: 

In [29]:
params = {'key1': 'value1', 'key2': ['value2', 'value3']}
r = requests.get('http://httpbin.org/get', params=params)

In [30]:
r.json()

{'args': {'key1': 'value1', 'key2': ['value2', 'value3']},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.12.4'},
 'origin': '92.242.59.6',
 'url': 'http://httpbin.org/get?key2=value2&key2=value3&key1=value1'}

In [31]:
r.text

'{\n  "args": {\n    "key1": "value1", \n    "key2": [\n      "value2", \n      "value3"\n    ]\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.12.4"\n  }, \n  "origin": "92.242.59.6", \n  "url": "http://httpbin.org/get?key2=value2&key2=value3&key1=value1"\n}\n'

Это все что я вспомнил про HTTP и его использование посредством requests. 

Еще несколько примеров испоользования requests, например, для получения изображений:

In [34]:
from PIL import Image
from io import BytesIO

r = requests.get("http://httpbin.org/image/png")
i = Image.open(BytesIO(r.content))

In [35]:
i.show()

Пример использования метода POST для записи формы на сервер.

In [40]:
r = requests.post("http://httpbin.org/post", data=params)
print(r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "key1": "value1", 
    "key2": [
      "value2", 
      "value3"
    ]
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "35", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.12.4"
  }, 
  "json": null, 
  "origin": "37.146.84.95", 
  "url": "http://httpbin.org/post"
}



Пример ошибки и её выброса:

In [41]:
r = requests.get("http://httpbin.org/status/404", data=params)
print(r.status_code)

404


In [42]:
r.raise_for_status()

HTTPError: 404 Client Error: NOT FOUND for url: http://httpbin.org/status/404

In [46]:
# немного синтаксиса как поолучать заголовки (по названию)
r.headers
r.headers['server']
# r.headers.get('server')

'nginx'

Так же можно выбрасывать ошибку, если привышен лимит времени на ответ:

In [66]:
r = requests.get('http://vk.com', timeout=1) #0.1 will raise the exception